In [58]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

In [59]:
#reading csv files into python
Artist_File_Path='Resources/artistDf.csv'
Billboard_File_Path='Resources/billboardHot100_1999-2019_sample.csv'
Grammy_File_Path='Resources/grammySongs_1999-2019.csv'

Adf=pd.read_csv(Artist_File_Path)
billboard_df = pd.read_csv(Billboard_File_Path)
Gdf=pd.read_csv(Grammy_File_Path)


In [60]:
def login():
    pw=input("Please enter your password for PGAdmin: ")
    return pw

In [61]:
#eliminate extra columns
Adf=Adf[['Artist','Followers','Genres','NumAlbums','Gender']].copy()
billboard_transformed = billboard_df[['Artists', 'Name', 'Peak.position', 'Weeks.on.chart','Week','Genre']].copy()
Gdf=Gdf[['Artist','GrammyAward','GrammyYear','Name', 'Genre']].copy()

In [62]:
Gdf

,Artist,GrammyAward,GrammyYear,Name,Genre
0,Childish Gambino,Record Of The Year,2018,this is America,General
1,Childish Gambino,Song Of The Year,2018,this is America,General
2,Lady Gaga,Best Pop Solo Performance,2018,Joanne (where Do you Think You're Goin'?),Pop
3,Lady Gaga & Bradley Cooper,Best Pop Duo/Group Performance,2018,Shallow,Pop
4,Silk City & Dua Lipa Featuring Diplo & Mark Ro...,Best Dance Recording,2018,Electricity,Dance/Electronic Music
...,...,...,...,...,...
423,Eminem,Best Rap Solo Performance,1999,My Name is,Rap
424,The roots featuring Erykah Badu,Best Rap Performance By A Duo Or Group,1999,you Got Me,Rap
425,Shania Twain,Best Female Country Vocal Performance,1999,Man! i Feel Like a Woman!,Country
426,George Jones,Best Male Country Vocal Performance,1999,Choices,Country


In [63]:
#get rid of all na columns
Adf=Adf.dropna()
Bdf=Bdf.dropna()
Gdf=Gdf.dropna()
#changed all strings to lower case
Gdf = Gdf.astype(str).apply(lambda x: x.str.lower())
Adf = Adf.astype(str).apply(lambda x: x.str.lower())
Bdf = Bdf.astype(str).apply(lambda x: x.str.lower())

In [64]:
# Rename the column headers
Adf = Adf.rename(columns={'Artist':'artist', 
'Genres': 'genre',
'NumAlbums': 'num_albums',
'Gender': 'gender',
'Group.Solo': 'group_solo'})

# Clean the data by dropping duplicates and setting the index
Adf.drop_duplicates('artist', inplace=True)
Adf.set_index('artist', inplace=True)
Adf.head()

,Followers,genre,num_albums,gender
artist,,,,
ed sheeran,52698756,"pop,uk pop",8,m
justin bieber,30711450,"canadian pop,dance pop,pop,post-teen pop",10,m
jonas brothers,3069527,"boy band,dance pop,pop,post-teen pop",10,m
drake,41420478,"canadian hip hop,canadian pop,hip hop,pop rap,...",11,m
chris brown,9676862,"dance pop,pop,pop rap,r&b,rap",6,m


In [65]:
billboard_transformed['Week'] = pd.to_datetime(billboard_transformed['Week'], format = '%Y-%m-%d').map(lambda x: x.strftime('%Y'))

# Rename the column headers
billboard_transformed = billboard_transformed.rename(columns={'Artists':'artist',
                                                              'Name': 'name_of_song',
                                                              'Peak.position': 'peak_position',
                                                              'Weeks.on.chart': 'weeks_on_chart',
                                                              'Week': 'year',
                                                              'Genre': 'genre'})

# Transform all strings to lower case
billboard_transformed = billboard_transformed.astype(str).apply(lambda x: x.str.lower())


for index in range(len(billboard_transformed)):
    # Assign artists into a variable
    artist_name = billboard_transformed['artist'][index]
    # Split artists
    artists_split = artist_name.split(', ')
    
    # Check if the 'artists_split' list is greater than 1. If it is, it means that more than one artist is listed for a song.
    if (len(artists_split)>1):
        # Modify original df billboard_transformed with one name in artist colum
        billboard_transformed['artist'][index]=artists_split[0]
        # Store row to be splitted in multiple rows in read_row (used to create the row for billboard_transformed_artist_split)
        read_row = billboard_transformed.iloc[index]
        
        # read each artist in the list and create a row for him/sher
        for index_artist in range(len(artists_split)):
            billboard_transformed.loc[len(billboard_transformed.index)] = [artists_split[index_artist], 
                                                                           read_row['name_of_song'],
                                                                           read_row['peak_position'],
                                                                           read_row['weeks_on_chart'], 
                                                                           read_row['year'],
                                                                           read_row['genre']]


#Keep only first row when there are artist/song duplicate
billboard_transformed.drop_duplicates(['artist', 'name_of_song'], keep='first', inplace=True)
billboard_transformed

,artist,name_of_song,peak_position,weeks_on_chart,year,genre
0,"lil nas,",old town road,1.0,7.0,2019,"country,atlanta,alternative country,hip-hop,tr..."
1,shawn mendes,senorita,nan,nan,2019,pop
2,billie eilish,bad guy,2.0,13.0,2019,"hip-hop,dark pop,house,trap,memes,alternative ..."
3,khalid,talk,3.0,20.0,2019,"synth-pop,pop"
4,ed sheeran,i don't care,2.0,7.0,2019,"canada,uk,dance,dance-pop,pop"
...,...,...,...,...,...,...
30220,justin timberlake,love never felt so good,9.0,20.0,2014,"eighties,soul,disco,dance,remix,r&;b,pop"
30229,lil jon,turn down for what,4.0,37.0,2014,"screen,soundtrack,memes,electronic,pop,rap,pro..."
30235,chris lake,delirious,90.0,3.0,2014,"dance,pop,electronic"
30236,tujamo,delirious,90.0,3.0,2014,"dance,pop,electronic"


In [66]:
# Rename the column headers
Gdf = Gdf.rename(columns={'Artist':'artist',
                                                                    'GrammyAward': 'grammy_award',
                                                                    'GrammyYear': 'grammy_year',
                                                                    'Name': 'name_of_song',
                                                                    'Genre': 'genre'})

# Transform all strings to lower case
Gdf = Gdf.astype(str).apply(lambda x: x.str.lower())
Gdf = Gdf.dropna()
Gdf = Gdf.reset_index()

del Gdf['index']

# Module used to split string with different delimiters
import re

for index, songs in Gdf.iterrows():
    # Get artist names
    artists = songs['artist']
    # Remove unnecessary strings
    artists = artists.replace('Songwriters','')
    artists = artists.replace('Songwriter','')
    artists = artists.replace('Artists','')
    artists = artists.replace('Artist','')
    artists = artists.replace('Soloists','')
    artists = artists.replace('Soloist','')
    artists = artists.replace('(','')
    artists = artists.replace(')','')
    artists = artists.replace(u'\xa0', u'')
    
    # Create list of artists
    artists_list = re.split("& |; |, |and |Featuring |with |featuring ",artists)
    
    for artist_index in range(len(artists_list)):
        # Remove with spaces in front and back for artist names
        artists_list[artist_index] = artists_list[artist_index].strip()
        
        # Remove blank names that we created as a result of the strip function above
        if artists_list[artist_index] == "":
            artists_list.pop(artist_index)
            
    # Update grammy_songs_transformed with spitted artists
    if len(artists_list)>1:
        Gdf.iloc[index,0]=artists_list[0]
        for name in artists_list:
            Gdf.loc[len(Gdf.index)]=[name,
                                                                               songs['grammy_award'],
                                                                                songs['grammy_year'],
                                                                               songs['name_of_song'],
                                                                               songs['genre']]

Gdf = Gdf.drop_duplicates(['grammy_award','name_of_song','artist'], keep='first')
Gdf

,artist,grammy_award,grammy_year,name_of_song,genre
0,childish gambino,record of the year,2018,this is america,general
1,childish gambino,song of the year,2018,this is america,general
2,lady gaga,best pop solo performance,2018,joanne (where do you think you're goin'?),pop
3,lady gaga,best pop duo/group performance,2018,shallow,pop
4,silk city,best dance recording,2018,electricity,dance/electronic music
...,...,...,...,...,...
768,rob thomas,record of the year,1999,smooth,general
770,rob thomas,song of the year,1999,smooth,general
772,rob thomas,best pop collaboration with vocals,1999,smooth,pop
774,everlast,best rock performance by a duo or group with v...,1999,put your lights on,rock


In [67]:
import psycopg2
pw=login()
connection = None
try:
    # In PostgreSQL, default username is 'postgres' and password is 'postgres'.
    # And also there is a default database exist named as 'postgres'.
    # Default host is 'localhost' or '127.0.0.1'
    # And default port is '54322'.
    connection= psycopg2.connect(
      database='postgres', user='postgres', password=pw, host='127.0.0.1', port= '5432'
    )
    print('Database connected.')

except:
    print('Database not connected.')

if connection is not None:
    connection.autocommit = True

    cur = connection.cursor()
    cur.execute("SELECT datname FROM pg_database;")

    list_database = cur.fetchall()
    if ('artist_db',) in list_database:
        print(" Database artist_db already exist")
    else:
        print(" Database artist_db does not exist, creating one right now.......")
        sql = '''CREATE database artist_db'''
        cur.execute(sql)
        print("Database created successfully........")
    connection.close()
    print('Done')


Database connected.
 Database artist_db does not exist, creating one right now.......
Database created successfully........
Done


In [71]:

rds_connection_string = f"postgres:{pw}@localhost:5432/artist_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [72]:
engine.table_names()

[]

In [73]:
Adf.to_sql(name='artists', con=engine, if_exists='append', index=False)
billboard_transformed.to_sql(name='billboard', con=engine, if_exists='append', index=False)
Gdf.to_sql(name='grammy_songs', con=engine, if_exists='append', index=False)
